In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [12]:
output_data_file = "weather_data.csv"
# output_data_file

types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=["City_ID.1","City_ID.1.1","City_ID.1.1.1"])
weather_data_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,386,sao felix do xingu,100,BR,1558378837,92,-6.64,-51.99,80.34,3.74
1,65,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
2,85,talnakh,75,RU,1558378783,92,69.49,88.39,15.80,29.08
3,139,porto novo,20,BR,1558378783,53,-23.68,-45.44,77.00,4.70
4,423,zyryanovsk,100,KZ,1558378588,88,49.74,84.27,48.84,1.81
...,...,...,...,...,...,...,...,...,...,...
495,387,yichun,89,CN,1558378781,98,47.72,128.88,41.64,3.58
496,437,christchurch,1,NZ,1558378744,100,-43.53,172.64,51.01,3.36
497,378,umtata,0,ZA,1558378835,51,-31.59,28.78,60.80,17.22
498,346,tabas,100,IR,1558378798,18,33.60,56.92,89.60,9.17


In [17]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

In [19]:
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.
locations = weather_data_df[["Lat", "Lng"]]

humidity = weather_data_df["Humidity"].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

print(fig)

Figure(layout=FigureLayout(height='420px'))


In [28]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.
perfect_weather = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (perfect_weather["Max Temp"] <= 85) & (perfect_weather["Max Temp"] > 75)
wind_speed = perfect_weather["Wind Speed"] < 10
cloudiness = perfect_weather["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
perfect_weather[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
11,cidreira,76.56,5.50,0
115,yabrud,80.60,8.05,0
165,arraial do cabo,84.20,4.70,0
411,gunjur,80.60,9.17,0
462,morondava,79.08,9.06,0


In [31]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""


hotel_df

,City,Country,Lat,Lng,Hotel Name
0,sao felix do xingu,BR,-6.64,-51.99,
1,qaanaaq,GL,77.48,-69.36,
2,talnakh,RU,69.49,88.39,
3,porto novo,BR,-23.68,-45.44,
4,zyryanovsk,KZ,49.74,84.27,
...,...,...,...,...,...
495,yichun,CN,47.72,128.88,
496,christchurch,NZ,-43.53,172.64,
497,umtata,ZA,-31.59,28.78,
498,tabas,IR,33.60,56.92,
